In [312]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [313]:
import gensim 
import re
import multiprocessing
import os
import numpy as np
from collections import Counter
import json
import nltk

In [314]:
print(gensim.__version__)

0.12.1


In [315]:
gensim.models.word2vec.FAST_VERSION

1

In [316]:
DATADIR = "/data/datasets/a2/wiki_dump/"

In [318]:
def parse_text(t):
    s = re.sub(r'[^\w\s]', '', t)
    r = s.lower().split()
    return r

In [320]:
def iter_text(directory):
    for fname in os.listdir(directory):
        with open(os.path.join(directory, fname), encoding="utf-8", errors="ignore") as f:
            if "json" in fname:
                d = json.load(f)
            else:
                d = f.read()
            for l in " ".join(d).split("\n"):
                r = parse_text(l)
                if len(r) > 0:
                    yield r

In [321]:
for i, t in enumerate(iter_text(DATADIR)):
    print(t)
    if i > 10:
        break

['39', 'queen', 'song', 'reasons', 'this', 'message', 'may', 'be', 'displayed']
['reasons', 'this', 'message', 'may', 'be', 'displayed']
['39', 'queen', 'song']
['reasons', 'this', 'message', 'may', 'be', 'displayed']
['and', 'star', 'power', 'reasons', 'this', 'message', 'may', 'be', 'displayed']
['reasons', 'this', 'message', 'may', 'be', 'displayed']
['and', 'star', 'power']
['reasons', 'this', 'message', 'may', 'be', 'displayed']
['13bisphosphoglyceric', 'acid', '13bisphosphoglyceric', 'acid', '13bisphosphoglycerate', 'or', '13bpg', 'is', 'a', '3carbon', 'organic', 'molecule', 'present', 'in', 'most', 'if', 'not', 'all', 'living', 'organisms', 'it', 'primarily', 'exists', 'as', 'a', 'metabolic', 'intermediate', 'in', 'both', 'glycolysis', 'during', 'respiration', 'and', 'the', 'calvin', 'cycle', 'during', 'photosynthesis', '13bpg', 'is', 'a', 'transitional', 'stage', 'between', 'glycerate', '3phosphate', 'and', 'glyceraldehyde', '3phosphate', 'during', 'the', 'fixationreduction', '

In [322]:
class Sentences(object):
    
    def __init__(self, directory):
        self.directory = directory
        
    def __iter__(self):
        for text in iter_text(self.directory):
            yield text

In [323]:
sentences = Sentences(DATADIR)

In [324]:
multiprocessing.cpu_count()

8

In [ ]:
%%time
word_model = gensim.models.Word2Vec(sentences, workers=multiprocessing.cpu_count(),
                                    size=300, iter=10, window=100,
                                    min_count=10, sg=1)

CPU times: user 4d 14h 2min 44s, sys: 23.6 s, total: 4d 14h 3min 8s
Wall time: 13h 49min 5s


In [ ]:
word_model.most_similar("acid")

In [ ]:
a = 1

In [337]:
len(word_model.vocab)

33747

In [338]:
TRAIN_SET = "/data/datasets/a2/training_set.tsv"

In [339]:
def quantify_text(t, model):
    words = parse_text(t)
#     return [w for w in words if w in model.vocab]
    emb = [word_model[w] for w in words if w in word_model.vocab and len(w) > 0]
    if emb != []:
        return np.mean(emb, axis=0)
    else:
        return np.zeros(model.vector_size)

In [340]:
from gensim import matutils
def similarity(v1, v2):
    return np.dot(matutils.unitvec(np.array(v1)), matutils.unitvec(np.array(v2)))

In [341]:
def range_answers(q, answers, models):
    scores = []
    for model in models:
        question = quantify_text(q, model)
        if (question == 0).all():
            return None
        ps = Counter()
        scores_model = []
        for a in answers:
            a_q = quantify_text(a, model)
            if (a_q == 0).all():
                scores_model.append(0) 
            else:
                si = similarity(question, a_q)
                scores_model.append(si)
        scores.append(scores_model)
    return np.mean(scores, axis=0)

In [352]:
tries_min = []
tries_max = []
tries_o = []
with open(TRAIN_SET) as f:
    next(f)
    for i, l in enumerate(f):
        [qid, q, r, aa, ab, ac, ad] = l.strip().split("\t")
        scores = range_answers(q, [aa, ab, ac, ad],
                               [word_model])
        no_scores = (scores == 0).all()
        if not no_scores:
            guess_min = "ABCD"[np.argmin(scores)]
            guess_max = "ABCD"[np.argmax(scores)]
            guess_o = "ABCD"[np.argmax(scores - np.mean(scores))]
        else:
            print(q, aa, ab, ac, ad)
#         print(q, aa, ab, ac, ad, scores, r, guess)
#         if i > 10:
#             break
        tries_max.append(1 if guess_max == r else 0)
        tries_min.append(1 if guess_min == r else 0)
        tries_o.append(1 if guess_o == r else 0)
#         if guess_min == r:
#             print(q, r, aa, ab, ac, ad, scores)

A sperm that contains alleles HqT fuses with an egg that contains alleles hqt. Which of the following genotypes will form in the offspring? HHqqTt HhQqTt Hhqqtt HhqqTt


In [353]:
np.mean(tries_max)

0.2944

In [354]:
np.mean(tries_min)

0.218

In [355]:
np.mean(tries_o)

0.2944

In [249]:
len(tries)

2500

In [62]:
tries[0:10]

[0, 1, 1, 0, 0, 0, 0, 0, 0, 1]

In [63]:
np.mean(word_model["dwarf"])

0.00052217481

In [64]:
np.mean(word_model["dog"])

-0.0066524623

In [65]:
len(word_model.vocab)

33612

In [257]:
# %%time
# tries = []
# with open(TRAIN_SET) as f:
#     next(f)
#     for i, l in enumerate(f):
#         [qid, q, r, aa, ab, ac, ad] = l.strip().split("\t")
#         scores = range_answers_p(q, [aa, ab, ac, ad])
#         guess = "ABCD"[scores.index(max(scores))]
# #         print(q, aa, ab, ac, ad, scores, r, guess)
# #         if i > 10:
# #             break
#         tries.append(1 if guess == r else 0)

In [258]:
np.mean(tries)

/data/anaconda3/lib/python3.4/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


nan

In [60]:
# # export model
# with open("/data/datasets/w2v_a2.tsv", "w") as fo:
#     for w in word_model.vocab:
#         fo.write("%s\t%s\n" % (w, "\t".join(map(str, word_model[w]))))

In [259]:
VALIDATION_SET = "/data/datasets/a2/validation_set.tsv"
SUBMISSION = "/data/datasets/a2/submission.txt"

In [262]:
with open(SUBMISSION, "w") as s:
    s.write("id,correctAnswer\n")
    with open(VALIDATION_SET) as fv:
        next(fv)
        for l in fv:
            [qid, q, aa, ab, ac, ad] = l.strip().split("\t")
            scores = range_answers(q, [aa, ab, ac, ad], [word_model])
            guess = "ABCD"[np.argmax(scores)]
            s.write("%s,%s\n" % (qid, guess))